# Opioid Study Data Collection
#### Defined study plan: https://docs.google.com/document/d/1kkTbheDP5vS8rh_C6W1U7qzthkScqrzKBjd0jSY3j48/edit
The main pieces consist of:

1. Inclusion/Exclusion Criteria
2. Demographic Data
3. Clinical Data

#### About: 
- Below I use PostgresSQL to query MIMIC-3 through a PostgresSQL adapter for Python
- Each step for collection is outlined below and the steps build on eachother
- Each step will have some relevant info to give context. In the SQL code, if the line starts with "--" it's a readable comment for more clarification on what the code does
- Once I write the SQL code, it is rendered as a Pandas dataframe, which is a really flexible library for data manipulation. From here, I can export the data to a common output format (.csv, .xslm, .tsv, etc.)

In [27]:
# Import libraries
import sys
import os
working_dir = os.path.abspath('.')
notebook_dir = os.path.join(working_dir, '../scripts')
assert os.path.exists(notebook_dir)
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from IPython.display import display, HTML # used to print out pretty pandas dataframes
display(HTML("<style>.container { width:100% !important; }</style>")) # widest display
import matplotlib.dates as dates
import finddrugs
import matplotlib.lines as mlines
import queries
import finddrugs_refactor as finder
import notebook_helper as helper

pd.options.display.max_colwidth = 1000
pd.options.display.width = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100


%matplotlib inline
plt.style.use('ggplot') 

# specify user/password/where the database is
sqluser = 'eightiesfanjan'
sqlpass = 'squiggle'
dbname = 'mimic'
schema_name = 'mimiciii'
host = 'localhost'

query_schema = 'SET search_path to ' + schema_name + ';'

# connect to the database
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=host)

In [89]:
reload(queries)
reload(helper)
reload(finder)

<module 'finddrugs_refactor' from '/Users/eightiesfanjan/Desktop/research/opioid_mimic_research/notebooks/../scripts/finddrugs_refactor.pyc'>

In [102]:
outputfile = '../data/phaseone_v1.csv'
df = pd.read_csv(outputfile, index_col='row_id')

In [103]:
df

,Unnamed: 0,subject_id,hadm_id,intime,outtime,age,diff_death_admit_hrs,diff_last_outtime,icd9_codes,seq_num,long_titles,valid_icu_admit,valid_age,valid_death,opiate_abuse,has_anoxic_brain,has_cancer,hist_found,opiate_history,admit_found,dis_found,group,opiates,hydromorphone,hydrocodone,oxycodone,morphine,fentanyl,tramadol,buprenorphine,methadone,oxymorphone,meperidine,icu_los_hours,hospital_intime,hospital_outtime,deathtime,hospital_expire_flag,admission_type,discharge_location,diagnosis,hospital_los_hours,gender,dod,dod_hosp,dod_ssn,death_days_since_hospital,30day_mortality,1year_mortality,admit_icd9,admit_long_titles,icu_los_days,hospital_los_days,icu_death,hos_death
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0,3,145834,2101-10-20 19:10:11,2101-10-26 20:43:09,76.52,5668.830278,NaN,"['0389', '78559', '5849', '4275', '41071', '4280', '6826', '4254', '2639']","[1, 2, 3, 4, 5, 6, 7, 8, 9]","['Unspecified septicemia', 'Other shock without mention of trauma', 'Acute kidney failure, unspecified', 'Cardiac arrest', 'Subendocardial infarction, initial episode of care', 'Congestive heart failure, unspecified', 'Cellulitis and abscess of leg, except foot', 'Other primary cardiomyopathies', 'Unspecified protein-calorie malnutrition']",1,1,1,0,0,0,1,0,1,1,3,1,0,0,1,0,0,0,0,0,0,0,145.549444,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,0,EMERGENCY,SNF,HYPOTENSION,258.833333,M,2102-06-14,NaN,2102-06-14 00:00:00,225.418056,0,1,0389,Unspecified septicemia,6.064560,10.784722,0,0
3,1,4,185777,2191-03-16 00:29:31,2191-03-17 16:46:31,47.84,NaN,NaN,"['042', '1363', '7994', '2763', '7907', '5715', '04111', 'V090', 'E9317']","[1, 2, 3, 4, 5, 6, 7, 8, 9]","['Human immunodeficiency virus [HIV] disease', 'Pneumocystosis', 'Cachexia', 'Alkalosis', 'Bacteremia', 'Cirrhosis of liver without mention of alcohol', 'Methicillin susceptible Staphylococcus aureus in conditions classified elsewhere and of unspecified site', 'Infection with microorganisms resistant to penicillins', 'Antiviral drugs causing adverse effects in therapeutic use']",1,1,1,0,0,0,1,0,1,1,2,0,0,0,0,0,0,0,0,0,0,0,40.283333,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,0,EMERGENCY,HOME WITH HOME IV PROVIDR,"FEVER,DEHYDRATION,FAILURE TO THRIVE",186.216667,F,NaN,NaN,NaN,NaN,0,0,042,Human immunodeficiency virus [HIV] disease,1.678472,7.759028,0,0
5,2,6,107064,2175-05-30 21:30:54,2175-06-03 13:39:54,65.94,NaN,NaN,"['40391', '9972', '2767', '2859', '2753', 'V1582']","[1, 3, 5, 6, 7, 8]","['Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease', 'Peripheral vascular complications, not elsewhere classified', 'Hyperpotassemia', 'Anemia, unspecified', 'Disorders of phosphorus metabolism', 'Personal history of tobacco use']",1,1,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,88.150000,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,0,ELECTIVE,HOME HEALTH CARE,CHRONIC RENAL FAILURE/SDA,392.750000,F,NaN,NaN,NaN,NaN,0,0,40391,"Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease",3.672917,16.364583,0,0
13,4,13,143045,2167-01-08 18:44:25,2167-01-12 10:43:31,39.86,NaN,NaN,"['41401', '4111', '25000', '4019', '2720']","[1, 2, 3, 4, 5]","['Coronary atherosclerosis of native coronary artery', 'Intermediate coronary syndrome', 'Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled', 'Unspecified essential hypertension', 'Pure hypercholesterolemia']",1,1,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,87.985000,2167-01-08 18:43:00,2167-01-15 15:15:00,NaN,0,EMERGENCY,HOME HEALTH CARE,CORONARY ARTERY DISEASE,164.533333,F,NaN,NaN,NaN,NaN,0,0,41401,Coronary atherosclerosis of native coronary artery,3.666042,6.855556,0,0
15,5,17,194023,2134-12-27 16:21:48,2134-12-29 18:04:03,47.45,NaN,NaN,"['7455', '45829', 'V1259', '2724']","[1, 2, 3, 4]","['Ostium secundum type atrial septal defect', 'Other iatrogenic hypotension', 'Personal history of other dise

In [111]:
tmp_filter = '0389' in df.icd9_codes

df["tmp"] = np.where('0389' in df.icd9_codes, 1, 0)

In [112]:
df

,Unnamed: 0,subject_id,hadm_id,intime,outtime,age,diff_death_admit_hrs,diff_last_outtime,icd9_codes,seq_num,long_titles,valid_icu_admit,valid_age,valid_death,opiate_abuse,has_anoxic_brain,has_cancer,hist_found,opiate_history,admit_found,dis_found,group,opiates,hydromorphone,hydrocodone,oxycodone,morphine,fentanyl,tramadol,buprenorphine,methadone,oxymorphone,meperidine,icu_los_hours,hospital_intime,hospital_outtime,deathtime,hospital_expire_flag,admission_type,discharge_location,diagnosis,hospital_los_hours,gender,dod,dod_hosp,dod_ssn,death_days_since_hospital,30day_mortality,1year_mortality,admit_icd9,admit_long_titles,icu_los_days,hospital_los_days,icu_death,hos_death,tmp
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0,3,145834,2101-10-20 19:10:11,2101-10-26 20:43:09,76.52,5668.830278,NaN,"['0389', '78559', '5849', '4275', '41071', '4280', '6826', '4254', '2639']","[1, 2, 3, 4, 5, 6, 7, 8, 9]","['Unspecified septicemia', 'Other shock without mention of trauma', 'Acute kidney failure, unspecified', 'Cardiac arrest', 'Subendocardial infarction, initial episode of care', 'Congestive heart failure, unspecified', 'Cellulitis and abscess of leg, except foot', 'Other primary cardiomyopathies', 'Unspecified protein-calorie malnutrition']",1,1,1,0,0,0,1,0,1,1,3,1,0,0,1,0,0,0,0,0,0,0,145.549444,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,0,EMERGENCY,SNF,HYPOTENSION,258.833333,M,2102-06-14,NaN,2102-06-14 00:00:00,225.418056,0,1,0389,Unspecified septicemia,6.064560,10.784722,0,0,0
3,1,4,185777,2191-03-16 00:29:31,2191-03-17 16:46:31,47.84,NaN,NaN,"['042', '1363', '7994', '2763', '7907', '5715', '04111', 'V090', 'E9317']","[1, 2, 3, 4, 5, 6, 7, 8, 9]","['Human immunodeficiency virus [HIV] disease', 'Pneumocystosis', 'Cachexia', 'Alkalosis', 'Bacteremia', 'Cirrhosis of liver without mention of alcohol', 'Methicillin susceptible Staphylococcus aureus in conditions classified elsewhere and of unspecified site', 'Infection with microorganisms resistant to penicillins', 'Antiviral drugs causing adverse effects in therapeutic use']",1,1,1,0,0,0,1,0,1,1,2,0,0,0,0,0,0,0,0,0,0,0,40.283333,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,0,EMERGENCY,HOME WITH HOME IV PROVIDR,"FEVER,DEHYDRATION,FAILURE TO THRIVE",186.216667,F,NaN,NaN,NaN,NaN,0,0,042,Human immunodeficiency virus [HIV] disease,1.678472,7.759028,0,0,0
5,2,6,107064,2175-05-30 21:30:54,2175-06-03 13:39:54,65.94,NaN,NaN,"['40391', '9972', '2767', '2859', '2753', 'V1582']","[1, 3, 5, 6, 7, 8]","['Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease', 'Peripheral vascular complications, not elsewhere classified', 'Hyperpotassemia', 'Anemia, unspecified', 'Disorders of phosphorus metabolism', 'Personal history of tobacco use']",1,1,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,88.150000,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,0,ELECTIVE,HOME HEALTH CARE,CHRONIC RENAL FAILURE/SDA,392.750000,F,NaN,NaN,NaN,NaN,0,0,40391,"Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease",3.672917,16.364583,0,0,0
13,4,13,143045,2167-01-08 18:44:25,2167-01-12 10:43:31,39.86,NaN,NaN,"['41401', '4111', '25000', '4019', '2720']","[1, 2, 3, 4, 5]","['Coronary atherosclerosis of native coronary artery', 'Intermediate coronary syndrome', 'Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled', 'Unspecified essential hypertension', 'Pure hypercholesterolemia']",1,1,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,87.985000,2167-01-08 18:43:00,2167-01-15 15:15:00,NaN,0,EMERGENCY,HOME HEALTH CARE,CORONARY ARTERY DISEASE,164.533333,F,NaN,NaN,NaN,NaN,0,0,41401,Coronary atherosclerosis of native coronary artery,3.666042,6.855556,0,0,0
15,5,17,194023,2134-12-27 16:21:48,2134-12-29 18:04:03,47.45,NaN,NaN,"['7455', '45829', 'V1259', '2724']","[1, 2, 3, 4]","['Ostium secundum type atrial septal defect', 'Other iatrogenic hypotension', 'Personal history 

## Phase 2: Demographic Data
Steps: 
    1. Acquire ICD9 codes for all comorbities
    2. Construct flags based on all specified comorbidities

From here on out we are now joining raw sql to our med list 

### Step 1) Acquire ICD9 codes for all comorbities

In [76]:
reload(queries)
df_tmp = helper.run_query(query=queries.play, db_connection=con, check_events=False)
df_tmp

,row_id,icd9_code,short_title,long_title
0,2186,27800,Obesity NOS,"Obesity, unspecified"
1,2187,27801,Morbid obesity,Morbid obesity


In [87]:
import icd9codes
reload(icd9codes)

In [90]:
helper.run_query(queries.comobordities(), con, False)

,row_id,icd9_code,short_title,long_title,has_copd,has_diabetes,has_cad,has_chfs_systolic,has_chf_diastolic,has_renal_disease,has_liver_disease,has_stroke,has_obesity,has_depression
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bacilli not found by bacteriological or histological examination, but tuberculosis confirmed by other methods [inoculation of animals]",0,0,0,0,0,0,0,0,0,0
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified",0,0,0,0,0,0,0,0,0,0
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or histological examination not done",0,0,0,0,0,0,0,0,0,0
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or histological examination unknown (at present)",0,0,0,0,0,0,0,0,0,0
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli found (in sputum) by microscopy",0,0,0,0,0,0,0,0,0,0
5,179,01174,TB pneumothorax-cult dx,"Tuberculous pneumothorax, tubercle bacilli not found (in sputum) by microscopy, but found by bacterial culture",0,0,0,0,0,0,0,0,0,0
6,180,01175,TB pneumothorax-histo dx,"Tuberculous pneumothorax, tubercle bacilli not found by bacteriological examination, but tuberculosis confirmed histologically",0,0,0,0,0,0,0,0,0,0
7,181,01176,TB pneumothorax-oth test,"Tuberculous pneumothorax, tubercle bacilli not found by bacteriological or histological examination, but tuberculosis confirmed by other methods [inoculation of animals]",0,0,0,0,0,0,0,0,0,0
8,182,01180,Pulmonary TB NEC-unspec,"Other specified pulmonary tuberculosis, unspecified",0,0,0,0,0,0,0,0,0,0
9,183,01181,Pulmonary TB NEC-no exam,"Other specified pulmonary tuberculosis, bacteriological or histological examination not done",0,0,0,0,0,0,0,0,0,0


In [95]:
comorbs_cols = [x for x in df_comorbs.columns if x not in ['short_title', 'long_title', 'row_id']]
df_comorbs = helper.run_query(queries.comobordities(), con, False)
df_comorbs = df_comorbs[comorbs_cols]

In [99]:
df_comorbs

,icd9_code,has_copd,has_diabetes,has_cad,has_chfs_systolic,has_chf_diastolic,has_renal_disease,has_liver_disease,has_stroke,has_obesity,has_depression
0,01166,0,0,0,0,0,0,0,0,0,0
1,01170,0,0,0,0,0,0,0,0,0,0
2,01171,0,0,0,0,0,0,0,0,0,0
3,01172,0,0,0,0,0,0,0,0,0,0
4,01173,0,0,0,0,0,0,0,0,0,0
5,01174,0,0,0,0,0,0,0,0,0,0
6,01175,0,0,0,0,0,0,0,0,0,0
7,01176,0,0,0,0,0,0,0,0,0,0
8,01180,0,0,0,0,0,0,0,0,0,0
9,01181,0,0,0,0,0,0,0,0,0,0


In [107]:
df_join = df.merge(df_comorbs, left_on='admit_icd9', right_on='icd9_code', how="left")

In [108]:
df_join

,Unnamed: 0,subject_id,hadm_id,intime,outtime,age,diff_death_admit_hrs,diff_last_outtime,icd9_codes,seq_num,long_titles,valid_icu_admit,valid_age,valid_death,opiate_abuse,has_anoxic_brain,has_cancer,hist_found,opiate_history,admit_found,dis_found,group,opiates,hydromorphone,hydrocodone,oxycodone,morphine,fentanyl,tramadol,buprenorphine,methadone,oxymorphone,meperidine,icu_los_hours,hospital_intime,hospital_outtime,deathtime,hospital_expire_flag,admission_type,discharge_location,diagnosis,hospital_los_hours,gender,dod,dod_hosp,dod_ssn,death_days_since_hospital,30day_mortality,1year_mortality,admit_icd9,admit_long_titles,icu_los_days,hospital_los_days,icu_death,hos_death,icd9_code,has_copd,has_diabetes,has_cad,has_chfs_systolic,has_chf_diastolic,has_renal_disease,has_liver_disease,has_stroke,has_obesity,has_depression
0,0,3,145834,2101-10-20 19:10:11,2101-10-26 20:43:09,76.52,5668.830278,NaN,"['0389', '78559', '5849', '4275', '41071', '4280', '6826', '4254', '2639']","[1, 2, 3, 4, 5, 6, 7, 8, 9]","['Unspecified septicemia', 'Other shock without mention of trauma', 'Acute kidney failure, unspecified', 'Cardiac arrest', 'Subendocardial infarction, initial episode of care', 'Congestive heart failure, unspecified', 'Cellulitis and abscess of leg, except foot', 'Other primary cardiomyopathies', 'Unspecified protein-calorie malnutrition']",1,1,1,0,0,0,1,0,1,1,3,1,0,0,1,0,0,0,0,0,0,0,145.549444,2101-10-20 19:08:00,2101-10-31 13:58:00,NaN,0,EMERGENCY,SNF,HYPOTENSION,258.833333,M,2102-06-14,NaN,2102-06-14 00:00:00,225.418056,0,1,0389,Unspecified septicemia,6.064560,10.784722,0,0,0389,0,0,0,0,0,0,0,0,0,0
1,1,4,185777,2191-03-16 00:29:31,2191-03-17 16:46:31,47.84,NaN,NaN,"['042', '1363', '7994', '2763', '7907', '5715', '04111', 'V090', 'E9317']","[1, 2, 3, 4, 5, 6, 7, 8, 9]","['Human immunodeficiency virus [HIV] disease', 'Pneumocystosis', 'Cachexia', 'Alkalosis', 'Bacteremia', 'Cirrhosis of liver without mention of alcohol', 'Methicillin susceptible Staphylococcus aureus in conditions classified elsewhere and of unspecified site', 'Infection with microorganisms resistant to penicillins', 'Antiviral drugs causing adverse effects in therapeutic use']",1,1,1,0,0,0,1,0,1,1,2,0,0,0,0,0,0,0,0,0,0,0,40.283333,2191-03-16 00:28:00,2191-03-23 18:41:00,NaN,0,EMERGENCY,HOME WITH HOME IV PROVIDR,"FEVER,DEHYDRATION,FAILURE TO THRIVE",186.216667,F,NaN,NaN,NaN,NaN,0,0,042,Human immunodeficiency virus [HIV] disease,1.678472,7.759028,0,0,042,0,0,0,0,0,0,0,0,0,0
2,2,6,107064,2175-05-30 21:30:54,2175-06-03 13:39:54,65.94,NaN,NaN,"['40391', '9972', '2767', '2859', '2753', 'V1582']","[1, 3, 5, 6, 7, 8]","['Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease', 'Peripheral vascular complications, not elsewhere classified', 'Hyperpotassemia', 'Anemia, unspecified', 'Disorders of phosphorus metabolism', 'Personal history of tobacco use']",1,1,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,88.150000,2175-05-30 07:15:00,2175-06-15 16:00:00,NaN,0,ELECTIVE,HOME HEALTH CARE,CHRONIC RENAL FAILURE/SDA,392.750000,F,NaN,NaN,NaN,NaN,0,0,40391,"Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease",3.672917,16.364583,0,0,40391,0,0,0,0,0,0,0,0,0,0
3,4,13,143045,2167-01-08 18:44:25,2167-01-12 10:43:31,39.86,NaN,NaN,"['41401', '4111', '25000', '4019', '2720']","[1, 2, 3, 4, 5]","['Coronary atherosclerosis of native coronary artery', 'Intermediate coronary syndrome', 'Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled', 'Unspecified essential hypertension', 'Pure hypercholesterolemia']",1,1,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,87.985000,2167-01-08 18:43:00,2167-01-15 15:15:00,NaN,0,EMERGENCY,HOME HEALTH CARE,CORONARY ARTERY DISEASE,164.533333,F,NaN,NaN,NaN,NaN,0,0,41401,Coronary atherosclerosis of native coronary artery,3.666042,6.855556,0,0,41401,0,0,1,0,0,0,0,0,0,0
4,5,17,194023,2134-12-27 16:21:48,2134-12-2